In [2]:
# Cell 1: Imports
import os
import random
import shutil
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance

In [3]:
balanced_folder = './balanced_dataset'  
output_base = './preprocessed'  

os.makedirs(output_base, exist_ok=True)

preprocessing_types = [
    'resized_100x100',
    'resized_150x150', 
    'normalized',
    'augmented_rotation',
    'augmented_flip',
    'augmented_zoom',
    'augmented_brightness',
    'contrast_clahe',
    'segmented'
]

for preprocess_type in preprocessing_types:
    preprocess_folder = os.path.join(output_base, preprocess_type)
    os.makedirs(preprocess_folder, exist_ok=True)
    
    for label in os.listdir(balanced_folder):
        label_path = os.path.join(balanced_folder, label)
        if os.path.isdir(label_path):
            os.makedirs(os.path.join(preprocess_folder, label), exist_ok=True)

In [4]:
all_images = []
for label in os.listdir(balanced_folder):
    label_path = os.path.join(balanced_folder, label)
    if os.path.isdir(label_path):
        for img_file in os.listdir(label_path):
            if img_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                all_images.append((label, img_file))


display_samples = {preprocess: [] for preprocess in preprocessing_types}

print(f"Found {len(all_images)} images to process")

Found 31810 images to process


## Image Preprocessing Implementation

The following cell applies all preprocessing techniques to each image.

In [ ]:
for label, img_filename in all_images:
    img_path = os.path.join(balanced_folder, label, img_filename)
    
    pil_img = Image.open(img_path)
    
    cv_img = cv2.imread(img_path)
    if cv_img is None:
        print(f"Warning: Could not read {img_path} with OpenCV. Skipping.")
        continue
    
    cv_img_rgb = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    
    img_resized_100 = pil_img.resize((100, 100), Image.LANCZOS)
    resized_100_path = os.path.join(output_base, 'resized_100x100', label, img_filename)
    img_resized_100.save(resized_100_path)
    
    img_resized_150 = pil_img.resize((150, 150), Image.LANCZOS)
    resized_150_path = os.path.join(output_base, 'resized_150x150', label, img_filename)
    img_resized_150.save(resized_150_path)
    
    
    cv_img_norm = cv_img_rgb / 255.0
    cv_img_norm_save = (cv_img_norm * 255).astype(np.uint8)
    norm_path = os.path.join(output_base, 'normalized', label, img_filename)
    cv2.imwrite(norm_path, cv2.cvtColor(cv_img_norm_save, cv2.COLOR_RGB2BGR))
    
    
    angle = random.uniform(-30, 30)
    height, width = cv_img_rgb.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width/2, height/2), angle, 1)
    cv_img_rotated = cv2.warpAffine(cv_img_rgb, rotation_matrix, (width, height))
    rot_path = os.path.join(output_base, 'augmented_rotation', label, img_filename)
    cv2.imwrite(rot_path, cv2.cvtColor(cv_img_rotated, cv2.COLOR_RGB2BGR))
    
    cv_img_flipped = cv2.flip(cv_img_rgb, 1)  
    flip_path = os.path.join(output_base, 'augmented_flip', label, img_filename)
    cv2.imwrite(flip_path, cv2.cvtColor(cv_img_flipped, cv2.COLOR_RGB2BGR))
    
    zoom_factor = random.uniform(0.8, 1.2)
    h, w = cv_img_rgb.shape[:2]
    zoom_matrix = cv2.getRotationMatrix2D((w/2, h/2), 0, zoom_factor)
    cv_img_zoomed = cv2.warpAffine(cv_img_rgb, zoom_matrix, (w, h))
    zoom_path = os.path.join(output_base, 'augmented_zoom', label, img_filename)
    cv2.imwrite(zoom_path, cv2.cvtColor(cv_img_zoomed, cv2.COLOR_RGB2BGR))
    
    brightness_factor = random.uniform(0.7, 1.3)
    enhancer = ImageEnhance.Brightness(pil_img)
    pil_img_bright = enhancer.enhance(brightness_factor)
    bright_path = os.path.join(output_base, 'augmented_brightness', label, img_filename)
    pil_img_bright.save(bright_path)
    
    img_gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_clahe = clahe.apply(img_gray)
    
    img_clahe_bgr = cv2.cvtColor(img_clahe, cv2.COLOR_GRAY2BGR)
    clahe_path = os.path.join(output_base, 'contrast_clahe', label, img_filename)
    cv2.imwrite(clahe_path, img_clahe_bgr)
    
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [largest_contour], 0, 255, -1)
        
        segmented = cv2.bitwise_and(cv_img, cv_img, mask=mask)
        seg_path = os.path.join(output_base, 'segmented', label, img_filename)
        cv2.imwrite(seg_path, segmented)
    
    for preprocess in preprocessing_types:
        
        if len(display_samples[preprocess]) < 5:
            if preprocess == 'segmented' and not contours:
                continue  
            display_samples[preprocess].append((label, img_filename))
        
        elif random.random() < 0.2:  
            if preprocess == 'segmented' and not contours:
                continue  
            replace_idx = random.randint(0, 4)
            display_samples[preprocess][replace_idx] = (label, img_filename)

print("Preprocessing complete. All images have been processed and saved to:", output_base)

In [ ]:
for preprocess in preprocessing_types:
    plt.figure(figsize=(20, 4))
    plt.suptitle(f'Preprocessing: {preprocess}', fontsize=16)
    
    for i, (label, img_filename) in enumerate(display_samples[preprocess], 1):
        img_path = os.path.join(output_base, preprocess, label, img_filename)
        
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            plt.subplot(1, 5, i)
            plt.imshow(img)
            plt.title(f'{label}\n{img_filename}')
            plt.axis('off')
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()